In [ ]:
#Two channal Model for cyberbully detection Channel1-(BERT_Embeddingv+ CNN),  Channel2 - (VecMap+CNN)

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers  as ppb 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from tensorflow.keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from tensorflow.keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
#from tensorflow.keras.models import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
import keras.backend as k
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
from tensorflow.keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from smart_open import smart_open
import datetime 
#from keras.utils import multi_gpu_model

import os
import time
import gc
import re
import random


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import logging
logging.basicConfig(level=logging.INFO)
from tensorflow import keras
import keras
import keras.utils
from keras import utils as np_utils

In [ ]:
data = pd.read_csv("Code_mixed_bully_dataset.csv", sep = ',')
data

In [ ]:
import seaborn as sns
ax=sns.countplot(data.Bully_label)

In [ ]:
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head(5)
data['processed_tweets'] = data['processed_tweets'].fillna('').apply(str)
df_text_genre = data[['processed_tweets', 'Bully_label']]
data.head(7)

train, test = train_test_split(df_text_genre, test_size=0.15, random_state=42, shuffle=True)

In [ ]:
test

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
    print("\n")  
    print("Confusion Matrix:\n ",confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy :\n ",accuracy_score(y_test,y_pred)*100) 
      
    print("Report : \n", classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, y_pred)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, y_pred)
    print('F1 score: %f' % f1)

In [ ]:
import copy
#clean_tweets = copy.deepcopy(tweets)
def clean_data(tweet):
    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

def tokenize(tweet):
    lower_tweet = tweet.lower()
    return word_tokenize(lower_tweet)
import nltk
nltk.download('punkt')
from tqdm import tqdm
import re
from nltk.tokenize import word_tokenize
tqdm.pandas(desc="Cleaning Data Phase I...")
train['Text_data'] = train['Text_data'].progress_apply(clean_data)
test['Text_data'] = test['Text_data'].progress_apply(clean_data)

tqdm.pandas(desc="Tokenizing Data...")
clean_tweets_train['tokens'] = train['Text_data'].progress_apply(tokenize)
clean_tweets_test['tokens'] = test['Text_data'].progress_apply(tokenize)

#train_vector = clean_tweets_train['tokens'].tolist()
#test_vector = clean_tweets_test['tokens'].tolist()
#train_labels = train['label'].values.tolist()
#test_labels = test['label'].values.tolist()

In [ ]:
#loc = "cc.en.300.bin"
loc = "/DATA/krishanu_2021cs19/krishanu/nlp/Hindi_English/bilingual_embedding/Hin_Eng_align_RCSLS.txt"
#loc = "Paper_Mikel Artetxe_Hindi_English_align_supervised.txt"
#loc = "VecMap_Mikel Artetxe_cc_fasttext_supervised_Hindi_English_align.txt"
#loc = "cc.hi.300.bin"
#loc = "/DATA/krishanu_2021cs19/krishanu/nlp/Hindi_English/bilingual_embedding/cc.en.300.bin"
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(loc, binary=False)
from gensim.models.wrappers import FastText
#print("Hi1")
#model = FastText.load_fasttext_format(loc)
#print("Hi2")

def get_embedding(word):
    try:
        embedding=model[word]
    except:
        embedding=np.zeros((300,))
    return embedding

In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
x_train2 = []
sentences = list(train['processed_tweets'])
for sen in sentences:
    x_train2.append(preprocess_text(sen))

In [ ]:
x_test2 = []
sentences = list(test['processed_tweets'])
for sen in sentences:
    x_test2.append(preprocess_text(sen))

In [ ]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(x_train2)

x_train = tokenizer.texts_to_sequences(x_train2)
x_test = tokenizer.texts_to_sequences(x_test2)

In [ ]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 30

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [ ]:
vocab_size

In [ ]:
import pickle

# saving
with open('tokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer1.pickle', 'rb') as handle:
    tokenizer2 = pickle.load(handle)

In [ ]:
sample = ['ram jane tera']
sam1 = tokenizer2.texts_to_sequences(sample)
sam1
sam2 = pad_sequences(sam1, padding='post', maxlen=maxlen)
sam2

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import AveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
import keras.backend as k
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

In [ ]:
embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = get_embedding(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
import numpy
import pickle
pickle.dump(embedding_matrix, open("embedding_matrix1.pkl", "wb"))

In [ ]:
def get_model(model_url, max_seq_length):
     inputs = dict(
     input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
  )
     
       # Define muril layer.
     muril_layer = hub.KerasLayer(model_url, trainable=False)
     outputs = muril_layer(inputs)
     assert 'pooled_output' in outputs
     assert 'sequence_output' in outputs
     sequence_output=outputs["sequence_output"]
     filters = 64
     convs1 = []
     filter_sizes = [1,2]
        
     
     #pooled_output, sequence_output = muril_layer([input_word_ids, input_mask, input_type_ids])
     #clf_output = sequence_output[:, 0, :]
     input1 = sequence_output
     for fsz in filter_sizes:
         l_conv = Conv1D(filters, kernel_size=fsz, activation='relu')(input1)
         #l_pool = MaxPooling1D(2)(l_conv)
         l_pool = AveragePooling1D(3)(l_conv)
         l_pool = Dropout(0.25)(l_pool)
         convs1.append(l_pool)
     l_merge1 = Concatenate(axis=1)(convs1)
     flat1 = Flatten()(l_merge1)
     drop1 = Dropout(0.4)(flat1)
     dense1 = Dense(60, activation='relu')(drop1)
     outputs = Dense(2, activation='softmax')(dense1)
     model = tf.keras.Model(inputs=[inputs], outputs=outputs)
     # compile
     model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
     # summarize
     print(model.summary())
     #plot_model(model, show_shapes=True, to_file='multichannel.png')
     return model, muril_layer
     

In [ ]:
os.environ["TFHUB_CACHE_DIR"] = '/tmp/tfhub'

#url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3"
#url = "https://tfhub.dev/google/MuRIL/1"
max_seq_length = 30
model, bert_layer = get_model(model_url= url, max_seq_length=max_seq_length)

In [ ]:
#from bert import bert_tokenization
import tokenization as bert_tokenization
#import bert_tokenizer as tokenizer
import numpy as np

vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
#from bert import bert_tokenization
import tokenization as bert_tokenization
#import bert_tokenizer as tokenizer
import numpy as np

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
 
def create_input(input_strings, tokenizer, max_seq_length):
    input_ids_all, input_mask_all, input_type_ids_all = [], [], []
    for input_string in input_strings:
        # Tokenize input.
        input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        sequence_length = min(len(input_ids), max_seq_length)
        # Padding or truncation.
        if len(input_ids) >= max_seq_length:
          input_ids = input_ids[:max_seq_length]
        else:
          input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

        input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

        input_ids_all.append(input_ids)
        input_mask_all.append(input_mask)
        input_type_ids_all.append([0] * max_seq_length)
    return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [ ]:
train_input = create_input(train.processed_tweets.values, tokenizer, max_seq_length=max_seq_length)
test_input = create_input(test.processed_tweets.values, tokenizer, max_seq_length=max_seq_length)
train_labels =  keras.utils.to_categorical(train.Bully_label.values, num_classes=2)
train_label = train.Bully_label.values
test_labels =  keras.utils.to_categorical(test.Bully_label.values, num_classes=2)
test_label = test.Bully_label.values

In [ ]:
weight_file = 'weight_mbert.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(weight_file, monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],#, earlystopping],
    batch_size=64,
    verbose=1)

In [ ]:
# define the channel 1 with our finetune
def define_model(length, vocab_size):
    # channel 1
    filters = 64
    convs1 = []
    filter_sizes = [1,2]
    
    #bert_input = Input(shape=(length,768))
    input0 = Input(shape=(30,768))
    #conv0 = Conv1D(filters=64, kernel_size=1, activation='relu')(input0)#, input_shape=(length,768))
    for fsz in filter_sizes:
        l_conv = Conv1D(filters, kernel_size=fsz, activation='relu')(input0)
        #l_pool = MaxPooling1D(2)(l_conv)
        l_pool = AveragePooling1D(3)(l_conv)
        l_pool = Dropout(0.25)(l_pool)
        convs1.append(l_pool)

    l_merge1 = Concatenate(axis=1)(convs1)
    #drop0 = Dropout(0.5)(conv0)
    #pool0 = MaxPooling1D(pool_size=2)(conv0)
    flat0 = Flatten()(l_merge1)  
    #channel 2
    convs2 = []
    inputs1 = Input(shape=(maxlen,))
    embedding1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)(inputs1)
    for fsz in filter_sizes:
        l_conv2 = Conv1D(filters, kernel_size=fsz, activation='relu')(embedding1)
        #l_pool2 = MaxPooling1D(2)(l_conv2)
        l_pool2 = AveragePooling1D(3)(l_conv2)
        l_pool2 = Dropout(0.25)(l_pool2)
        convs2.append(l_pool2)
    l_merge2 = Concatenate(axis=1)(convs2)
    #conv1 = Conv1D(filters=64, kernel_size =1, activation='relu')(embedding1)
    #drop1 = Dropout(0.5)(conv1)
    #pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(l_merge2)

    # merge
    #flat4 = Flatten()(pool3)
    merged = concatenate([flat0,flat1])
    drop4 = Dropout(0.5)(merged)
    # interpretation
    dense1 = Dense(60, activation='relu')(drop4)
    drop5 = Dropout(0.5)(dense1)
    #dense2 = Dense(40, activation='relu')(drop5)
    outputs = Dense(2, activation='sigmoid')(drop5)
    model = Model(inputs=[input0, inputs1], outputs=outputs)
    # compile
    model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
# define the channel 2 
def define_model(length, vocab_size):
    # channel 1
    filters = 64
    convs1 = []
    filter_sizes = [1,2]
    inputs1 = Input(shape=(maxlen,))
    embedding1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)(inputs1)
    for fsz in filter_sizes:
        l_conv2 = Conv1D(filters, kernel_size=fsz, activation='relu')(embedding1)
        #l_pool2 = MaxPooling1D(2)(l_conv2)
        l_pool2 = AveragePooling1D(3)(l_conv2)
        l_pool2 = Dropout(0.25)(l_pool2)
        convs1.append(l_pool2)
    l_merge2 = Concatenate(axis=1)(convs1)
    #conv1 = Conv1D(filters=64, kernel_size =1, activation='relu')(embedding1)
    #drop1 = Dropout(0.5)(conv1)
    #pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(l_merge2)

    # merge
    #flat4 = Flatten()(pool3)
    #merged = concatenate([flat0,flat1])
    drop4 = Dropout(0.4)(flat1)
    # interpretation
    dense1 = Dense(60, activation='relu')(drop4)
    drop5 = Dropout(0.5)(dense1)
    #dense2 = Dense(40, activation='relu')(drop5)
    outputs = Dense(2, activation='softmax')(drop5)
    model = Model(inputs=[inputs1], outputs=outputs)
    # compile
    model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
maxlen = 30
model = define_model(maxlen, vocab_size )

In [ ]:
model.fit(x_train, train_labels, epochs=35,validation_split=0.15, batch_size=64)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
# Saving the model for Future Inferences

model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")

In [ ]:
# imports

#from keras import model_from_json 

# opening and store file in a variable

json_file = open('model1.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("model1.h5")
print("Loaded Model from disk")

In [ ]:
y_pred1 = loaded_model.predict(sam2)
y_pred1
test_prediction1 = np.argmax(y_pred1, axis=-1)
test_prediction1


In [ ]:
y_pred1

In [ ]:
y_pred = loaded_model.predict(x_test)

In [ ]:

test_prediction = np.argmax(y_pred, axis=-1)
cal_accuracy(test_prediction,test_label)

In [ ]:
# define the two channel model
def define_model(length, vocab_size):
    # channel 1 + channel 2
    filters = 64
    convs1 = []
    filter_sizes = [1,2,3]
    
    #bert_input = Input(shape=(length,768))
    input0 = Input(shape=(30,768))
    #conv0 = Conv1D(filters=64, kernel_size=1, activation='relu')(input0)#, input_shape=(length,768))
    for fsz in filter_sizes:
        l_conv = Conv1D(filters, kernel_size=fsz, activation='relu')(input0)
        #l_pool = MaxPooling1D(2)(l_conv)
        l_pool = AveragePooling1D(3)(l_conv)
        l_pool = Dropout(0.25)(l_pool)
        convs1.append(l_pool)

    l_merge1 = Concatenate(axis=1)(convs1)
    #drop0 = Dropout(0.5)(conv0)
    #pool0 = MaxPooling1D(pool_size=2)(conv0)
    flat0 = Flatten()(l_merge1)  
    #channel 2
    convs2 = []
    inputs1 = Input(shape=(maxlen,))
    embedding1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)(inputs1)
    for fsz in filter_sizes:
        l_conv2 = Conv1D(filters, kernel_size=fsz, activation='relu')(embedding1)
        #l_pool2 = MaxPooling1D(2)(l_conv2)
        l_pool2 = AveragePooling1D(3)(l_conv2)
        l_pool2 = Dropout(0.25)(l_pool2)
        convs2.append(l_pool2)
    l_merge2 = Concatenate(axis=1)(convs2)
    #conv1 = Conv1D(filters=64, kernel_size =1, activation='relu')(embedding1)
    #drop1 = Dropout(0.5)(conv1)
    #pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(l_merge2)

    # merge
    #flat4 = Flatten()(pool3)
    merged = concatenate([flat0,flat1])
    drop4 = Dropout(0.5)(merged)
    # interpretation
    dense1 = Dense(60, activation='relu')(drop4)
    drop5 = Dropout(0.5)(dense1)
    #dense2 = Dense(40, activation='relu')(drop5)
    outputs = Dense(2, activation='sigmoid')(drop5)
    model = Model(inputs=[input0, inputs1], outputs=outputs)
    # compile
    model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
# define model
model1 = define_model(40, vocab_size )

In [ ]:
#model2 = define_model2(max_seq_length, vocab_size )

In [ ]:
model1.fit([train_embedding, x_train], train_labels, validation_split=0.15, epochs=10, batch_size=64)

In [ ]:
model1.save('model_bert_vecmap1.h5')

In [ ]:
from keras import models
model2 = models.load_model('model_bert_vecmap1.h5')
#model1 = keras.models.load_model('model_bert_vecmap.h5')

In [ ]:
#model2.fit([x_train], train_labels, epochs=10, batch_size=32)

In [ ]:
#testX = encode_text(tokenizer, test.Text_data.values, length)
#print(trainX.shape, testX.shape)

In [ ]:
#mbedding_matrix = np.random.random(300)
#mbedding_matrix

In [ ]:
# evaluate model on training dataset
loss, acc = model1.evaluate([train_embedding,x_train], train_labels, verbose=0)
print('Train Accuracy: %f' % (acc*100))
 
# evaluate model on test dataset dataset
loss, acc = model1.evaluate([test_embedding,x_test],test_labels, verbose=0)
print('Test Accuracy: %f' % (acc*100))
y_pred = model2.predict([test_embedding, x_test])

In [ ]:
test_prediction = np.argmax(y_pred, axis=-1)
test_prediction


In [ ]:
#test_prediction = np.where(y_pred > 0.5, 1, 0)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_label, test_prediction)

cm

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(test_label, test_prediction)) 
print('Report : ')
print(classification_report(test_label, test_prediction))

In [ ]:
cal_accuracy(test_label, test_prediction)

In [ ]:
#TO get maximum value as 1 and rest to zero
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

y_test=pd.DataFrame(test_labels)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

#print(result)
#print(predicted)

In [ ]:
print(classification_report(result, predicted))

In [ ]:
# With fine tunning
def get_two_channel_model_with_train(model_url, max_seq_length,vocab_size):
     input0 = dict(
     input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
  )
     
       # Define muril layer.
     muril_layer = hub.KerasLayer(model_url, trainable = False)
     outputs = muril_layer(input0)
     assert 'pooled_output' in outputs
     assert 'sequence_output' in outputs
     sequence_output=outputs["sequence_output"]
     #pooled_output, sequence_output = muril_layer([input_word_ids, input_mask, input_type_ids])
     clf_output = sequence_output
     embed_layer = Conv1D(filters=64, kernel_size=2, activation='relu')(clf_output)
     net = AveragePooling1D(4)(embed_layer)
     #gru = Bidirectional(GRU(64, activation='sigmoid', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(net)
     #capsule = Capsule(num_capsule=1 ,dim_capsule=72, routings=1,share_weights=True)(gru)
     flat0 = Flatten()(net)
     
     inputs1 = Input(shape=(maxlen,))
     embedding1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length , trainable=False)(inputs1)
     #for fsz in filter_sizes:
        #l_conv2 = Conv1D(filters, kernel_size=fsz, activation='relu')(embedding1)
        #l_pool2 = MaxPooling1D(2)(l_conv2)
        #l_pool2 = AveragePooling1D(3)(l_conv2)
        #l_pool2 = Dropout(0.25)(l_pool2)
        #convs2.append(l_pool2)
     #l_merge2 = Concatenate(axis=1)(convs2)
     conv1 = Conv1D(filters=64, kernel_size =2, activation='relu')(embedding1)
     drop1 = Dropout(0.5)(conv1)
     pool1 = MaxPooling1D(pool_size=2)(drop1)
     flat1 = Flatten()(pool1)

    # merge
    #flat4 = Flatten()(pool3)
     merged = concatenate([flat0,flat1])
     drop4 = Dropout(0.5)(merged)
    # interpretation
     dense1 = Dense(60, activation='relu')(drop4)
     drop5 = Dropout(0.5)(dense1)
     outputs = Dense(2, activation='softmax')(drop5)
     model = Model(inputs=[input0, inputs1], outputs=outputs)
    # compile
     model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize
     print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    #return model
     #output = Dense(2, activation='softmax')(capsule)
     
     


    
     #model = tf.keras.Model(inputs=inputs,outputs=output)
     #model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
     
     #assert 'sequence_output' in outputs
     
     #assert 'pooled_output' in outputs
     
     #assert 'default' in outputs
     
     #return tf.keras.Model(inputs=inputs,outputs=outputs["sequence_output"]), muril_layer
     return model, muril_layer

In [ ]:
def get_one_channel_model_with_train(model_url, max_seq_length,vocab_size):
     input0 = dict(
     input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
     input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
  )
     
       # Define muril layer.
     muril_layer = hub.KerasLayer(model_url, trainable = False)
     outputs = muril_layer(input0)
     assert 'pooled_output' in outputs
     assert 'sequence_output' in outputs
     sequence_output=outputs["sequence_output"]
     #pooled_output, sequence_output = muril_layer([input_word_ids, input_mask, input_type_ids])
     clf_output = sequence_output
     embed_layer = Conv1D(filters=64, kernel_size=2, activation='relu')(clf_output)
     net = AveragePooling1D(4)(embed_layer)
     #gru = Bidirectional(GRU(64, activation='sigmoid', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(net)
     #capsule = Capsule(num_capsule=1 ,dim_capsule=72, routings=1,share_weights=True)(gru)
     flat0 = Flatten()(net)
     
     #inputs1 = Input(shape=(maxlen,))
     #embedding1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length , trainable=False)(inputs1)
     #for fsz in filter_sizes:
        #l_conv2 = Conv1D(filters, kernel_size=fsz, activation='relu')(embedding1)
        #l_pool2 = MaxPooling1D(2)(l_conv2)
        #l_pool2 = AveragePooling1D(3)(l_conv2)
        #l_pool2 = Dropout(0.25)(l_pool2)
        #convs2.append(l_pool2)
     #l_merge2 = Concatenate(axis=1)(convs2)
     #conv1 = Conv1D(filters=64, kernel_size =2, activation='relu')(embedding1)
     #drop1 = Dropout(0.5)(conv1)
     #pool1 = MaxPooling1D(pool_size=2)(drop1)
     #flat1 = Flatten()(pool1)

    # merge
    #flat4 = Flatten()(pool3)
     #merged = concatenate([flat0,flat1])
     drop4 = Dropout(0.5)(flat0)
    # interpretation
     dense1 = Dense(60, activation='relu')(drop4)
     drop5 = Dropout(0.5)(dense1)
     outputs = Dense(2, activation='softmax')(drop5)
     model = Model(inputs=[input0], outputs=outputs)
    # compile
     model.compile(tf.keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize
     print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    #return model
     #output = Dense(2, activation='softmax')(capsule)
     
     


    
     #model = tf.keras.Model(inputs=inputs,outputs=output)
     #model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
     
     #assert 'sequence_output' in outputs
     
     #assert 'pooled_output' in outputs
     
     #assert 'default' in outputs
     
     #return tf.keras.Model(inputs=inputs,outputs=outputs["sequence_output"]), muril_layer
     return model, muril_layer

In [ ]:
os.environ["TFHUB_CACHE_DIR"] = '/tmp/tfhub'
#url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
#url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3"
url = "https://tfhub.dev/google/MuRIL/1"
max_seq_length = 30
model4, muril_layer = get_one_channel_model_with_train(
    model_url= url, max_seq_length=max_seq_length, vocab_size = vocab_size)

In [ ]:
model3, muril_layer = get_two_channel_model_with_train(
    model_url= url, max_seq_length=max_seq_length, vocab_size = vocab_size)

In [ ]:
#from bert import bert_tokenization
import tokenization as bert_tokenization
#import bert_tokenizer as tokenizer
import numpy as np

vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def create_input(input_strings, tokenizer, max_seq_length):
    input_ids_all, input_mask_all, input_type_ids_all = [], [], []
    for input_string in input_strings:
        # Tokenize input.
        input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        sequence_length = min(len(input_ids), max_seq_length)
        # Padding or truncation.
        if len(input_ids) >= max_seq_length:
          input_ids = input_ids[:max_seq_length]
        else:
          input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

        input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

        input_ids_all.append(input_ids)
        input_mask_all.append(input_mask)
        input_type_ids_all.append([0] * max_seq_length)
    return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [ ]:
max_len = 30
bert_train_input = create_input(train.Text_data.values, tokenizer, max_seq_length=max_len)
bert_test_input = create_input(test.Text_data.values, tokenizer, max_seq_length=max_len)
train_labels =  keras.utils.to_categorical(train.label.values, num_classes=2)
test_labels =  keras.utils.to_categorical(test.label.values, num_classes=2)

In [ ]:
bert_train_input

In [ ]:
model4.fit([bert_train_input], train_labels, epochs=20, batch_size=128)

In [ ]:
model3.fit([bert_train_input, x_train], train_labels, epochs=20, batch_size=32)

In [ ]:
# evaluate model on training dataset
loss, acc = model3.evaluate([bert_train_input,x_train], train_labels, verbose=0)
print('Train Accuracy: %f' % (acc*100))
 
# evaluate model on test dataset dataset
loss, acc = model3.evaluate([bert_test_input,x_test],test_labels, verbose=0)
print('Test Accuracy: %f' % (acc*100))
y_pred = model3.predict([bert_test_input, x_test])

In [ ]:
test_prediction = np.argmax(y_pred, axis=-1)
test_prediction


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_label, test_prediction)

cm

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(test_label, test_prediction)) 
print('Report : ')
print(classification_report(test_label, test_prediction))